In [25]:
import csv
from scipy.optimize import linprog as lp
import numpy as np


def read_data(file_path):
    matrix = list()
    vector = list()
    with open(file_path, newline='') as file:
        reader = csv.reader(file, delimiter=',', quotechar='|')
        next(reader)
        for row in reader:
            matrix_row = list()
            for i in range(0, 4):
                matrix_row.append(float(row[i]))
            matrix.append(matrix_row)
            vector.append(float(row[4]))
    return [matrix, vector]


def create_conditions_matrix(matrix):
    dimension = len(matrix[0])
    size = len(matrix)
    
    weight_matrix = np.matrix(matrix)
    identity_minus = -np.identity(size)
    
    first_part = np.concatenate((weight_matrix, identity_minus), axis = 1)
    second_part = np.concatenate((-weight_matrix, identity_minus), axis = 1)
    return np.concatenate((first_part, second_part), axis = 0)


def solve_linear(matrix, vector, tolerance):
    dimension = len(matrix[0]) # amount of real parameters
    size = len(matrix) # amount of fake "a" parameters
    
    # the coefficients of the function to minimize
    linear_coefficients = dimension*[0] + size*[1]
    
    # calculating the list of bounds for coefficients
    weight_bounds = (None, None)
    a_bounds = (0.0, None)
    parameters_bounds = dimension*[weight_bounds] + size*[a_bounds]
    
    # calculating conditions and their bounds
    conditions_bounds = np.concatenate((np.array(vector), -np.array(vector)))
    conditions_matrix = create_conditions_matrix(matrix).tolist()
    
    res = lp(linear_coefficients, A_ub=conditions_matrix, b_ub=conditions_bounds, bounds=parameters_bounds,
             options={"disp": True, "bland": True, "tol": tolerance})
    return res.x[:dimension] # we're not interested in fake parameters


[X, y] = read_data('basketball.csv')
solution = solve_linear(X, y, 0.001)

print("Weights:\n", solution)
print("Approximate solution:\n", np.dot(X, solution))

Optimization terminated successfully.
         Current function value: 208.953912  
         Iterations: 138
Weights:
 [ -2.83245223e+00   1.05564300e-02   3.07481879e+01   1.76187408e+01]
Approximate solution:
 [  8.54501445  11.47330651  11.30706286   8.6         12.28295085
  11.24258929  12.66690717  10.85504161  10.72443237  12.53401171
  10.04351411  11.53864838   8.1         11.49006792  11.55342856
  14.03256986  13.77975753   3.97684065  10.15687145  10.5          8.89213346
   8.4603416    2.99351803   9.20675361  11.925672    10.77536508
  11.91753771   8.33719612   9.94485897  12.27103568  13.87710405
   9.4553227   12.26904263   8.4603416    9.54156888  12.24862152  13.6
  12.25309555   6.00471451   5.36921069   8.20411419   6.32895091
  12.52148615  13.82168927  10.52106879  10.21767998  14.77604725
   8.98206128   8.91304465  13.9034563   12.55372344  10.28588545
   9.9930349    9.436993  ]
